In [12]:
from datasets import load_dataset
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer
from rlhfutils.data import preproc_wgpt, preproc_apf, preproc_hh

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
rlcd = load_dataset("csv", data_files="../rlcd-llama/simulated_data/simulated_preference_data_consolidated_helpful7b.csv")['train']

In [40]:
webgpt = load_dataset("openai/webgpt_comparisons", split="train")
webgpt = pd.DataFrame([preproc_wgpt(w) for w in webgpt])

Found cached dataset webgpt_comparisons (/home/prasann/.cache/huggingface/datasets/openai___webgpt_comparisons/default/0.0.0/8b5d5879cdc98c4c0099af6053dffe8d504588d43d3b11f1b1ec223ab1e8db0a)


In [46]:
stack = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/reward", split="train")
stack = stack.select(range(100000))
stack = pd.DataFrame(stack)

Found cached dataset parquet (/home/prasann/.cache/huggingface/datasets/lvwerra___parquet/lvwerra--stack-exchange-paired-1132fd48cbaac550/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [3]:
apfgpt = load_dataset("tatsu-lab/alpaca_farm", 'alpaca_gpt4_preference')['preference']
apfhum= load_dataset("tatsu-lab/alpaca_farm", 'alpaca_human_preference')['preference']
apfgpt = pd.DataFrame([preproc_apf(w) for w in apfgpt])
apfhum = pd.DataFrame([preproc_apf(w) for w in apfhum])

Found cached dataset alpaca_farm (/home/prasann/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_gpt4_preference/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.99it/s]
Found cached dataset alpaca_farm (/home/prasann/.cache/huggingface/datasets/tatsu-lab___alpaca_farm/alpaca_human_preference/1.0.0/79d38dc3f12abd62869e376303b68092e8385769e22f05166fe96a3dac29a57a)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 291.74it/s]


In [14]:
hh_train = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base", split="train")
hh_train = pd.DataFrame([preproc_hh(w) for w in hh_train])

Found cached dataset json (/home/prasann/.cache/huggingface/datasets/Anthropic___json/Anthropic--hh-rlhf-37c6f75e35564d2a/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


In [27]:
ind = 4
print(hh_train['response_k'][ind])
print("_____")
print(hh_train['response_j'][ind])

 That depends on the 9 year old, but if they like thrillers or mysteries then maybe something like “The Body in the Bog” or “The Boy Who Followed Ripley.” For more modern, female-written fiction I might recommend “Jane Eyre” or “Frankenstein.”

Human: That sounds cool

Assistant: Do you want some quotes?
_____
 That depends on the 9 year old, but if they like thrillers or mysteries then maybe something like “The Body in the Bog” or “The Boy Who Followed Ripley.” For more modern, female-written fiction I might recommend “Jane Eyre” or “Frankenstein.”

Human: That sounds cool

Assistant: I’m a programmer, so I’m really into good programming texts, like “The Practice of Programming” or “JavaScript: The Good Parts.” But that might be a bit too much for a 9 year old, so


In [8]:
tokenizer = AutoTokenizer.from_pretrained("/home/prasann/Projects/tfr-decoding/llama/llama")

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [9]:
# NOTE I validated that RLCD doesn't have any input formatted stuff
def rlcdmakeprefs (inpdf):
    allres = []
    for row in inpdf:
        res = {}
        res['question'] = row['instruction']
        if row['preference']==1:
            res['response_j'] = row['output_1']
            res['response_k'] = row['output_2']
        else:
            res['response_j'] = row['output_2']
            res['response_k'] = row['output_1']
        allres.append(res)
    return pd.DataFrame(allres).dropna().reset_index(drop=True)

# take in processed df, given tokenizer, tokenize everything
def tokall (pdf): 
    gtoks = []
    btoks = []
    for ind, row in pdf.iterrows():
        gtoks.append(len(tokenizer(row['response_j']).input_ids))
        btoks.append(len(tokenizer(row['response_k']).input_ids))
    pdf['gtoks'] = gtoks
    pdf['btoks'] = btoks
    return pdf

def lenbias (indf):
    return (indf['gtoks']>indf['btoks']).mean()

In [32]:
print("processing")
rlcproc = rlcdmakeprefs(rlcd)
print("tokenizing")
rlcproc = tokall(rlcproc)

processing
tokenizing


In [43]:
wgptproc = tokall(webgpt)

In [45]:
lenbias(wgptproc)

0.5565430585350905

In [51]:
stackproc = tokall(stack)

In [52]:
lenbias(stackproc)

0.59698

In [10]:
apfhumbproc = tokall(apfhum)
apfgptproc = tokall(apfgpt)

In [11]:
print(lenbias(apfhumbproc))
print(lenbias(apfgptproc))

0.5418429470642865
0.535024650780608


In [29]:
hhproc = tokall(hh_train)

In [30]:
lenbias(hhproc)

0.5828219459336147